## Question 3

In [ ]:
# Read data: -----------------------------------------------------------------
Demo1 = pd.read_sas('DEMO_G.XPT'); Demo1['cohort']='2011-2012'
Demo2 = pd.read_sas('DEMO_H.XPT'); Demo2['cohort']='2013-2014'
Demo3 = pd.read_sas('DEMO_I.XPT'); Demo3['cohort']='2015-2016'
Demo4 = pd.read_sas('DEMO_J.XPT'); Demo4['cohort']='2017-2018'

# Clean data: ----------------------------------------------------------------
Demo = Demo1.append(Demo2).append(Demo3).append(Demo4).reset_index(drop=True)
Demo = Demo[['SEQN','RIDAGEYR','RIDRETH3','DMDEDUC2','DMDMARTL','RIDSTATR',
             'SDMVPSU', 'SDMVSTRA', 'WTMEC2YR', 'WTINT2YR','cohort']]
Demo = Demo.fillna(-1) ## Transfer NaN to -1

# Convert data type: ---------------------------------------------------------
for i in ['SEQN','RIDAGEYR','RIDRETH3','DMDEDUC2','DMDMARTL','RIDSTATR',
             'SDMVPSU', 'SDMVSTRA']:
    Demo[i] = Demo[i].astype(int, errors='ignore')

# Remame the columns: --------------------------------------------------------
Demo = Demo.rename({'SEQN':'id','RIDAGEYR':'age','RIDRETH3':
                    'race and ethnicity', 'DMDEDUC2':'education',
                    'DMDMARTL':'marital status'},axis=1)

# Save the results: ----------------------------------------------------------
Demo.to_pickle('./Demo_pickle.pkl')

# 79: ------------------------------------------------------------------------

In [ ]:
# Read data: ------------------------------------------------------------------
OHX1 = pd.read_sas('OHXDEN_G.XPT'); OHX1['cohort']='2011-2012'
OHX2 = pd.read_sas('OHXDEN_H.XPT'); OHX2['cohort']='2013-2014'
OHX3 = pd.read_sas('OHXDEN_I.XPT'); OHX3['cohort']='2015-2016'
OHX4 = pd.read_sas('OHXDEN_J.XPT'); OHX4['cohort']='2017-2018'

# Clean data: ----------------------------------------------------------------
OHX = OHX1.append(OHX2).append(OHX3).append(OHX4).reset_index(drop=True)
OHX = pd.concat([OHX.iloc[:,:64].drop(['OHDEXSTS','OHXIMP'],axis=1),
                                                 OHX['cohort']],axis=1)
OHX = OHX.fillna(-1) ## Transfer NaN to -1

# Convert data type: ---------------------------------------------------------
for i in range(33):
    OHX[str(OHX.columns[i])] = OHX[str(OHX.columns[i])].astype(
        int, errors='ignore')

# Remame the columns: --------------------------------------------------------
OHX = OHX.rename({'SEQN':'id','OHDDESTS':'dentition status code'},axis=1)

# Save the results: ----------------------------------------------------------
OHX.to_pickle('./OHX_pickle.pkl')

# 79: ------------------------------------------------------------------------